# PreProcessing

In [1]:
import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum

In [2]:
df1=pd.read_csv('AllOdiPlayers.csv')
df2=pd.read_csv('AllOdicaptains.csv')
df3=pd.read_csv('AllTestcaptains.csv')
df4=pd.read_csv('AllTestplayers.csv')

In [3]:
col_a_df1 = df1['Name']
col_a_df2 = df2['Name']
col_a_df3 = df3['Name']
col_a_df4 = df4['Name']
indian_values = pd.concat([col_a_df1, col_a_df2, col_a_df3, col_a_df4])
unique_values = indian_values.unique()
unique_values_df = pd.DataFrame(unique_values, columns=['UniqueValues'])
unique_values_df


,UniqueValues
0,Syed Abid Ali
1,Bishan Singh Bedi
2,Farokh Engineer
3,Sunil Gavaskar
4,Madan Lal
...,...
383,Abhinav Mukund
384,Hanuma Vihari
385,Shahbaz Nadeem
386,Thangarasu Natarajan


In [4]:
bat_df = pd.read_csv("bat_india.csv")
ball_df = pd.read_csv("ball_india.csv")
match_df = pd.read_csv("match_india.csv")

In [18]:
len(match_df['match_detail_id'].unique())

670

In [5]:
# Extract india
players_df = unique_values_df['UniqueValues']
bat_india_players = bat_df[bat_df['name_x'].isin(players_df)]
ball_india_players = ball_df[ball_df['name_x'].isin(players_df)]
ball_india_players

,name_x,playing_role,id,order,run_conceded,maidens,wickets,overs,economy,wides,...,matchtype,series_id,match_detail_id,title,runs,over,run_rate,match_id,opp_team_id,team_id
0,Ajit Agarkar,BOWL,8544,2,51,0,2,10.0,5.10,3,...,odi,182,773,Bangladesh Innings,234,50.0,4.680000,633,2,7
7,Irfan Pathan,AR,8542,0,45,0,1,9.0,5.00,5,...,odi,182,773,Bangladesh Innings,234,50.0,4.680000,633,2,7
8,Joginder Sharma,AR,8543,1,28,2,1,8.0,3.50,1,...,odi,182,773,Bangladesh Innings,234,50.0,4.680000,633,2,7
9,Sourav Ganguly,BAT,8545,3,19,0,0,4.0,4.75,0,...,odi,182,773,Bangladesh Innings,234,50.0,4.680000,633,2,7
10,Harbhajan Singh,BOWL,8546,4,46,1,0,10.0,4.60,4,...,odi,182,773,Bangladesh Innings,234,50.0,4.680000,633,2,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9161,Jasprit Bumrah,BOWL,36252,1,19,3,0,6.0,3.17,0,...,test,27,4840,England inning,112,48.4,2.301370,3017,2,5
9162,Axar Patel,AR,36261,0,32,0,5,15.0,2.13,0,...,test,27,4840,England inning,81,30.4,2.641304,3017,2,5
9163,Ravichandran Ashwin,AR,36262,1,48,3,4,15.0,3.20,0,...,test,27,4840,England inning,81,30.4,2.641304,3017,2,5
9165,Washington Sundar,AR,36263,2,1,0,1,0.4,1.50,0,...,test,27,4840,England inning,81,30.4,2.641304,3017,2,5


In [6]:
bat_india_players.to_csv('bat_india_players.csv')
ball_india_players.to_csv('ball_india_players.csv')

In [7]:
ball_india=pd.read_csv('ball_india_players.csv')
bat_india=pd.read_csv('bat_india_players.csv')

In [8]:
# Normalizing the columns (you may adjust the method of normalization)
normalized_df_bat = bat_india.copy()
for col in ['strike_rate', 'fall_of_wicket_over', 'runs_y', 'run_rate', 'runs_x']:
    normalized_df_bat[col] = (bat_india[col] - bat_india[col].min()) / (bat_india[col].max() - bat_india[col].min())

# Creating a composite batting performance metric
normalized_df_bat['batting_score'] = (normalized_df_bat['strike_rate'] +
                                  normalized_df_bat['fall_of_wicket_over'] +
                                  normalized_df_bat['runs_y'] +
                                  normalized_df_bat['run_rate'] +
                                  normalized_df_bat['runs_x']) / 5
# Create column for "opponent_name"
normalized_df_bat['opponent_name'] = normalized_df_bat['name_y'].str.replace('v', '')  # removing 'v'
normalized_df_bat['opponent_name'] = normalized_df_bat['opponent_name'].str.replace('india', '', case=False)  # removing 'india'
normalized_df_bat['opponent_name'] = normalized_df_bat['opponent_name'].str.strip()  # trimming extra spaces

# Covert date and select just "YYYY-MM-DD"
normalized_df_bat['start_date'] = pd.to_datetime(normalized_df_bat['start_date'])
normalized_df_bat['date'] = normalized_df_bat['start_date'].dt.date

In [9]:
# Group by player to get average performance
average_batting_performance = normalized_df_bat.groupby('name_x')['batting_score'].mean()


In [10]:
# Normalizing the columns
normalized_df_ball = ball_india.copy()
for col in ['wickets', 'overs', 'economy', 'maidens', 'zeros', 'no_balls', 'wides']:
    normalized_df_ball[col] = (ball_india[col] - ball_india[col].min()) / (ball_india[col].max() - ball_india[col].min())

# Creating metrics
# Wickets per Over (WPO)
normalized_df_ball['WPO'] = normalized_df_ball['wickets'] / normalized_df_ball['overs']

# Economy Rate (ER) - Inverted because lower economy is better
normalized_df_ball['ER'] = 1 - normalized_df_ball['economy']

# Maiden Over Ratio (MOR)
normalized_df_ball['MOR'] = normalized_df_ball['maidens'] / normalized_df_ball['overs']

# Dot Ball Ratio (DBR)
normalized_df_ball['DBR'] = normalized_df_ball['zeros'] / (normalized_df_ball['overs'] * 6)

# No-Ball and Wide Penalty (NWP) - Inverted because fewer no-balls and wides are better
normalized_df_ball['NWP'] = 1 - (normalized_df_ball['no_balls'] + normalized_df_ball['wides']) / normalized_df_ball['overs']

# Creating a composite bowling performance metric
normalized_df_ball['bowling_score'] = (normalized_df_ball['WPO'] +
                                       normalized_df_ball['ER'] +
                                       normalized_df_ball['MOR'] +
                                      # normalized_df_ball['DBR'] +
                                       normalized_df_ball['NWP']) / 5
WPO, ER, MOR, NWP,

# Create column for "opponent_name"
normalized_df_ball['opponent_name'] = normalized_df_ball['name_y'].str.replace('v', '')  # removing 'v'
normalized_df_ball['opponent_name'] = normalized_df_ball['opponent_name'].str.replace('india', '', case=False)  # removing 'india'
normalized_df_ball['opponent_name'] = normalized_df_ball['opponent_name'].str.strip()  # trimming extra spaces

# Covert date and select just "YYYY-MM-DD"
normalized_df_ball['start_date'] = pd.to_datetime(normalized_df_ball['start_date'])
normalized_df_ball['date'] = normalized_df_ball['start_date'].dt.date

In [11]:
average_bowling_performance = normalized_df_ball.groupby('name_x')['bowling_score'].mean()


In [12]:
# Define sets of players
batsmen = set(normalized_df_bat[normalized_df_bat['playing_role'] == 'BAT']['name_x'])
bowlers = set(normalized_df_ball[normalized_df_ball['playing_role'] == 'BOWL']['name_x'])
all_rounders = set(normalized_df_bat[normalized_df_bat['playing_role'] == 'AR']['name_x']) | \
               set(normalized_df_ball[normalized_df_ball['playing_role'] == 'AR']['name_x'])


# Optimization Models

### Variables
- $x_{p}$ is a binary decision variable (1 if player p is chose, 0 if not)
- $t_{p}$ represents the batting score for player p
- $l_{p}$ represents the bowling score for player p
- $b_{p}$ represents the normalized batting score for player p
- $w_{p}$ represents the normalized bowling score for player p
- $T$ = set of batsmen
- $L$ = set of bowlers
- $R$ = set of all rounders
- $A$ = set of all players ($T \, \cup \, L \, \cup \, R$)

### Constraints:
- (1) Team cannot exceed 11 players
\begin{align*}
\sum_{p \, \in \, A} x_{p} = 11 \\
\end{align*}
- (2) Must be at least 4 batsmen on team
\begin{align*}
\sum_{p \, \in \, T} x_{p} \geq 4
\end{align*}
- (3) Must be at least 4 bowlers on team
\begin{align*}
\sum_{p \, \in \, L} x_{p} \geq 4 
\end{align*}
- (4) Cannot be more than 2 all rounders on team
\begin{align*}
\sum_{p \, \in \, R} x_{p} \leq 2
\end{align*}

### Objective function
#### Model 1
- Maximize team batting score
\begin{align*}
\max \quad \sum_{p \, \in \, T \, \cup \, R} x_{p} * t_{p}
\end{align*}

#### Model 2
- Maximize bowling score
\begin{align*}
\max \quad \sum_{p \, \in \, L \, \cup \, R} x_{p} * l_{p}
\end{align*}

#### Model 3
- Maximize combined normalized batting and normalized bowling scores
\begin{align*}
\max \quad \sum_{p \, \in \, T \, \cup \, R} x_{p} * b_{p} + \sum_{p \, \in \, L \, \cup \, R} x_{p} * w_{p}
\end{align*}

# Model 1 - Max batting

- $x_{p}$ is a binary decision variable (1 if player p is chose, 0 if not)
- $t_{p}$ represents the batting score for player p
- $T$ = set of batsmen
- $L$ = set of bowlers
- $R$ = set of all rounders
- $A$ = set of all players ($T \, \cup \, L \, \cup \, R$)

\begin{align*}
\max \quad \sum_{p \, \in \, T \, \cup \, R} x_{p} * t_{p}
\end{align*}

\begin{align*}
\text{st} \quad & (1) \quad \sum_{p \, \in \, A} x_{p} = 11 \\
& (2) \quad \sum_{p \, \in \, T} x_{p} \geq 4 \\
& (3) \quad \sum_{p \, \in \, L} x_{p} \geq 4 \\
& (4) \quad \sum_{p \, \in \, R} x_{p} \leq 2 \\
\end{align*}

Constraints:
- (1) Team cannot exceed 11 players
- (2) Must be at least 4 bastmen on team
- (3) Much be at least 4 bowlers on team
- (4) Cannot be more than 2 all rounders on team

In [28]:
def max_batting_perf(normalized_df_bat, normalized_df_ball, opponent_name=None, match_type=None, date=None):  
    
    # Filter data
    filtered_df_bat = normalized_df_bat.copy()
    filtered_df_ball = normalized_df_ball.copy()
    

    if opponent_name is not None:
        filtered_df_bat = filtered_df_bat[filtered_df_bat['opponent_name'] == opponent_name]
        filtered_df_ball = filtered_df_ball[filtered_df_ball['opponent_name'] == opponent_name]

    if match_type is not None:
        filtered_df_bat = filtered_df_bat[filtered_df_bat['matchtype'] == match_type]
        filtered_df_ball = filtered_df_ball[filtered_df_ball['matchtype'] == match_type]

    if date is not None:
        date = pd.to_datetime(date).date()
        filtered_df_bat = filtered_df_bat[filtered_df_bat['date'] == date]
        filtered_df_ball = filtered_df_ball[filtered_df_ball['date'] == date]   
        
    # Define sets of players
    batsmen = set(filtered_df_bat[filtered_df_bat['playing_role'] == 'BAT']['name_x'])
    bowlers = set(filtered_df_ball[filtered_df_ball['playing_role'] == 'BOWL']['name_x'])
    all_rounders = set(filtered_df_bat[filtered_df_bat['playing_role'] == 'AR']['name_x']) | \
                   set(filtered_df_ball[filtered_df_ball['playing_role'] == 'AR']['name_x'])
        
    # Create the optimization model
    model = Model()
    
    # Decision variables for each player
    player_vars = {}  

    # Filter players with NaN mean batting scores
    batsmen = {player for player in batsmen if not np.isnan(filtered_df_bat[filtered_df_bat['name_x'] == player]['batting_score'].mean())}
    bowlers = {player for player in bowlers if not np.isnan(filtered_df_bat[filtered_df_bat['name_x'] == player]['batting_score'].mean())}
    all_rounders = {player for player in all_rounders if not np.isnan(filtered_df_bat[filtered_df_bat['name_x'] == player]['batting_score'].mean())}

    # Create player_vars
    player_vars = {}
    for player in batsmen:
        player_vars[player] = model.addVar(vtype=GRB.BINARY, name=f"bat_{player}")
    for player in bowlers:
        player_vars[player] = model.addVar(vtype=GRB.BINARY, name=f"ball_{player}")
    for player in all_rounders:
        player_vars[player] = model.addVar(vtype=GRB.BINARY, name=f"ar_{player}")

    # Objective function: Maximize batting performance per player
    model.setObjective(
        quicksum(player_vars[player] * filtered_df_bat[filtered_df_bat['name_x'] == player]['batting_score'].mean() 
                 for player in batsmen | all_rounders),
        GRB.MAXIMIZE
    )


    # Constraints
    # Total team size: 11 players
    model.addConstr(quicksum(player_vars[player] for player in player_vars) == 11, "TotalTeamSize")

    # Minimum number of batsmen, e.g., 4
    min_batsmen = 4
    model.addConstr(quicksum(player_vars[player] for player in batsmen) >= min_batsmen, "MinBatsmen")

    # Minimum number of bowlers, e.g., 4
    min_bowlers = 4
    model.addConstr(quicksum(player_vars[player] for player in bowlers) >= min_bowlers, "MinBowlers")

    # Maximum number of all-rounders, e.g., 2
    max_all_rounders = 2
    model.addConstr(quicksum(player_vars[player] for player in all_rounders) <= max_all_rounders, "MaxAllRounders")

    # Solve the model
    model.optimize();
    
    # Find average batting score of team for each individual match
    mean_batting_score_per_match = filtered_df_bat.groupby('match_detail_id')['batting_score'].mean()
    
    # Find average batting score of match
    baseline_mean_batting_score = mean_batting_score_per_match.mean()
    print(f"Average team batting score for entire match: {baseline_mean_batting_score}")

    # Extracting the selected players
    selected_players = [player for player in player_vars if player_vars[player].X > 0.5]
    
    # Create df of selected players and their mean batting scores
    selected_players_df = pd.DataFrame(columns=['Player', 'Mean Batting Score'])
    for player in selected_players:
        mean_batting_score = filtered_df_bat[filtered_df_bat['name_x'] == player]['batting_score'].mean()
        selected_players_df = selected_players_df.append({'Player': player, 'Mean Batting Score': mean_batting_score}, ignore_index=True)

    # Find team average batting score
    optimal_mean_batting_score = selected_players_df['Mean Batting Score'].mean()

    print("Optimal team average batting score:", optimal_mean_batting_score)

    return selected_players, baseline_mean_batting_score, optimal_mean_batting_score

In [29]:
max_batting_perf(normalized_df_bat, normalized_df_ball, opponent_name = "Australia", match_type = "test")

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 41 columns and 82 nonzeros
Model fingerprint: 0x7b8d55ca
Variable types: 0 continuous, 41 integer (41 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 3e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 1e+01]
Found heuristic solution: objective 1.2095400
Presolve removed 1 rows and 16 columns
Presolve time: 0.00s
Presolved: 3 rows, 25 columns, 49 nonzeros
Found heuristic solution: objective 1.5363990
Variable types: 0 continuous, 25 integer (24 binary)

Root relaxation: cutoff, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0   

C:\Users\PC User\AppData\Local\Temp\ipykernel_39604\3655651861.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  selected_players_df = selected_players_df.append({'Player': player, 'Mean Batting Score': mean_batting_score}, ignore_index=True)
C:\Users\PC User\AppData\Local\Temp\ipykernel_39604\3655651861.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  selected_players_df = selected_players_df.append({'Player': player, 'Mean Batting Score': mean_batting_score}, ignore_index=True)
C:\Users\PC User\AppData\Local\Temp\ipykernel_39604\3655651861.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  selected_players_df = selected_players_df.append({'Player': player, 'Mean Batting Score': mean_batting_score}, ignore_index=T

(['Cheteshwar Pujara',
  'Suresh Raina',
  'Karun Nair',
  'Mayank Agarwal',
  'Sourav Ganguly',
  'Shardul Thakur',
  'Varun Aaron',
  'Jayant Yadav',
  'Navdeep Saini',
  'Ravindra Jadeja',
  'Washington Sundar'],
 0.2006692120722943,
 0.2036326473291841)

# Model 2 - Max bowling

- $x_{p}$ is a binary decision variable (1 if player p is chose, 0 if not)
- $l_{p}$ represents the bowling score for player p
- $T$ = set of batsmen
- $L$ = set of bowlers
- $R$ = set of all rounders
- $A$ = set of all players ($T \, \cup \, L \, \cup \, R$)

\begin{align*}
\max \quad \sum_{p \, \in \, L \, \cup \, R} x_{p} * l_{p}
\end{align*}

\begin{align*}
\text{st} \quad & (1) \quad \sum_{p \, \in \, A} x_{p} = 11 \\
& (2) \quad \sum_{p \, \in \, T} x_{p} \geq 4 \\
& (3) \quad \sum_{p \, \in \, L} x_{p} \geq 4 \\
& (4) \quad \sum_{p \, \in \, R} x_{p} \leq 2 \\
\end{align*}

Constraints:
- (1) Team cannot exceed 11 players
- (2) Must be at least 4 bastmen on team
- (3) Much be at least 4 bowlers on team
- (4) Cannot be more than 2 all rounders on team

In [21]:
# Function to impute missing values
def impute_missing_values(df):
    for col in df.columns:
        if df[col].dtype == 'object':  # If the column is of object type (string)
            # Impute with mode
            mode_value = df[col].mode()[0]
            df[col].fillna(mode_value, inplace=True)
        else:  # If the column is numeric
            # Impute with mean
            mean_value = df[col].mean()
            df[col].fillna(mean_value, inplace=True)

In [26]:
def max_bowling_perf(normalized_df_bat, normalized_df_ball, opponent_name=None, match_type=None, date=None):  
    
    # Filter data
    filtered_df_bat = normalized_df_bat.copy()
    filtered_df_ball = normalized_df_ball.copy()

    if opponent_name is not None:
        filtered_df_bat = filtered_df_bat[filtered_df_bat['opponent_name'] == opponent_name]
        filtered_df_ball = filtered_df_ball[filtered_df_ball['opponent_name'] == opponent_name]

    if match_type is not None:
        filtered_df_bat = filtered_df_bat[filtered_df_bat['matchtype'] == match_type]
        filtered_df_ball = filtered_df_ball[filtered_df_ball['matchtype'] == match_type]

    if date is not None:
        date = pd.to_datetime(date).date()
        filtered_df_bat = filtered_df_bat[filtered_df_bat['date'] == date]
        filtered_df_ball = filtered_df_ball[filtered_df_ball['date'] == date]   

    # Imput missing values
    impute_missing_values(filtered_df_ball)

    # Create the optimization model2
    model = Model()

    # Define sets of players
    batsmen = set(filtered_df_bat[filtered_df_bat['playing_role'] == 'BAT']['name_x'])
    bowlers = set(filtered_df_ball[filtered_df_ball['playing_role'] == 'BOWL']['name_x'])
    all_rounders = set(filtered_df_bat[filtered_df_bat['playing_role'] == 'AR']['name_x']) | \
                   set(filtered_df_ball[filtered_df_ball['playing_role'] == 'AR']['name_x'])
    
    # Decision variables for each player
    player_vars = {}  

    # Filter players with NaN mean bowling scores
    batsmen = {player for player in batsmen if not np.isnan(filtered_df_ball[filtered_df_ball['name_x'] == player]['bowling_score'].mean())}
    bowlers = {player for player in bowlers if not np.isnan(filtered_df_ball[filtered_df_ball['name_x'] == player]['bowling_score'].mean())}
    all_rounders = {player for player in all_rounders if not np.isnan(filtered_df_ball[filtered_df_ball['name_x'] == player]['bowling_score'].mean())}
    
    # Create player_vars
    player_vars = {}
    for player in batsmen:
        player_vars[player] = model.addVar(vtype=GRB.BINARY, name=f"bat_{player}")
    for player in bowlers:
        player_vars[player] = model.addVar(vtype=GRB.BINARY, name=f"ball_{player}")
    for player in all_rounders:
        player_vars[player] = model.addVar(vtype=GRB.BINARY, name=f"ar_{player}")

    # Objective function: Maximize bowling performance
    model.setObjective(
        quicksum(player_vars[player] * filtered_df_ball[filtered_df_ball['name_x'] == player]['bowling_score'].mean() 
                 for player in bowlers | all_rounders),
        GRB.MAXIMIZE
    )

    # Constraints
    # Total team size: 11 players
    model.addConstr(quicksum(player_vars[player] for player in player_vars) == 11, "TotalTeamSize")

    # Minimum number of batsmen, e.g., 4
    min_batsmen = 4
    model.addConstr(quicksum(player_vars[player] for player in batsmen) >= min_batsmen, "MinBatsmen")

    # Minimum number of bowlers, e.g., 4
    min_bowlers = 4
    model.addConstr(quicksum(player_vars[player] for player in bowlers) >= min_bowlers, "MinBowlers")

    # Maximum number of all-rounders, e.g., 2
    max_all_rounders = 2
    model.addConstr(quicksum(player_vars[player] for player in all_rounders) <= max_all_rounders, "MaxAllRounders")

    # Solve the model2
    model.optimize()

    # Extracting the selected players
    selected_players = [player for player in player_vars if player_vars[player].X > 0.5]
    
    # Find average bowling score of team for each individual match
    mean_bowling_score_per_match = filtered_df_ball.groupby('match_detail_id')['bowling_score'].mean()
    
    # Find average bowling score of match
    baseline_mean_bowling_score = mean_bowling_score_per_match.mean()
    print(f"Average team bowling score for entire match: {baseline_mean_bowling_score}")
    
    # Create df of selected players and their mean bowling scores
    selected_players_df = pd.DataFrame(columns=['Player', 'Mean Bowling Score'])
    for player in selected_players:
        mean_bowling_score = filtered_df_ball[filtered_df_ball['name_x'] == player]['bowling_score'].mean()
        selected_players_df = selected_players_df.append({'Player': player, 'Mean Bowling Score': mean_bowling_score}, ignore_index=True)

    # Find team average batting score
    optimal_mean_bowling_score = selected_players_df['Mean Bowling Score'].mean()

    print("Optimal team average bowling score:", optimal_mean_bowling_score)

    return selected_players, baseline_mean_bowling_score, optimal_mean_bowling_score

In [27]:
max_bowling_perf(normalized_df_bat, normalized_df_ball, opponent_name = "Australia", match_type = "test")

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 32 columns and 64 nonzeros
Model fingerprint: 0x2347a90d
Variable types: 0 continuous, 32 integer (32 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 7e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 1e+01]
Found heuristic solution: objective 3.6514343
Presolve removed 1 rows and 8 columns
Presolve time: 0.00s
Presolved: 3 rows, 24 columns, 47 nonzeros
Found heuristic solution: objective 4.6540757
Variable types: 0 continuous, 24 integer (23 binary)

Root relaxation: cutoff, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0    

C:\Users\PC User\AppData\Local\Temp\ipykernel_39604\383754574.py:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  selected_players_df = selected_players_df.append({'Player': player, 'Mean Bowling Score': mean_bowling_score}, ignore_index=True)
C:\Users\PC User\AppData\Local\Temp\ipykernel_39604\383754574.py:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  selected_players_df = selected_players_df.append({'Player': player, 'Mean Bowling Score': mean_bowling_score}, ignore_index=True)
C:\Users\PC User\AppData\Local\Temp\ipykernel_39604\383754574.py:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  selected_players_df = selected_players_df.append({'Player': player, 'Mean Bowling Score': mean_bowling_score}, ignore_index=True

(['Sachin Tendulkar',
  'Suresh Raina',
  'Virender Sehwag',
  'Murali Vijay',
  'Shardul Thakur',
  'Mohammed Siraj',
  'Kuldeep Yadav',
  'Mohammed Shami',
  'Jasprit Bumrah',
  'Ravindra Jadeja',
  'Ravichandran Ashwin'],
 0.5201055174098315,
 0.5784082316714588)

# Model 3 - Both

- $x_{p}$ is a binary decision variable (1 if player p is chose, 0 if not)
- $t_{p}$ represents the normalized batting score for player p
- $l_{p}$ represents the normalized bowling score for player p
- $T$ = set of batsmen
- $L$ = set of bowlers
- $R$ = set of all rounders
- $A$ = set of all players ($T \, \cup \, L \, \cup \, R$)

\begin{align*}
\max \quad \sum_{p \, \in \, T \, \cup \, R} x_{p} * t_{p} + \sum_{p \, \in \, L \, \cup \, R} x_{p} * l_{p}
\end{align*}

\begin{align*}
\text{st} \quad & (1) \quad \sum_{p \, \in \, A} x_{p} = 11 \\
& (2) \quad \sum_{p \, \in \, T} x_{p} \geq 4 \\
& (3) \quad \sum_{p \, \in \, L} x_{p} \geq 4 \\
& (4) \quad \sum_{p \, \in \, R} x_{p} \leq 2 \\
\end{align*}

Constraints:
- (1) Team cannot exceed 11 players
- (2) Must be at least 4 bastmen on team
- (3) Much be at least 4 bowlers on team
- (4) Cannot be more than 2 all rounders on team

In [22]:
def max_both_perf(normalized_df_bat, normalized_df_ball, opponent_name=None, match_type=None, date=None):
       
    # Filter data
    filtered_df_bat = normalized_df_bat.copy()
    filtered_df_ball = normalized_df_ball.copy()

    if opponent_name is not None:
        filtered_df_bat = filtered_df_bat[filtered_df_bat['opponent_name'] == opponent_name]
        filtered_df_ball = filtered_df_ball[filtered_df_ball['opponent_name'] == opponent_name]

    if match_type is not None:
        filtered_df_bat = filtered_df_bat[filtered_df_bat['matchtype'] == match_type]
        filtered_df_ball = filtered_df_ball[filtered_df_ball['matchtype'] == match_type]

    if date is not None:
        date = pd.to_datetime(date).date()
        filtered_df_bat = filtered_df_bat[filtered_df_bat['date'] == date]
        filtered_df_ball = filtered_df_ball[filtered_df_ball['date'] == date]
        actual_players = pd.concat([filtered_df_bat['name_x'], filtered_df_ball['name_x']]).drop_duplicates().reset_index(drop=True)
    
    # Imput missing values
    impute_missing_values(filtered_df_ball)
    
    # Create the optimization model
    model = Model()

    # Define sets of players
    batsmen = set(filtered_df_bat[filtered_df_bat['playing_role'] == 'BAT']['name_x'])
    bowlers = set(filtered_df_ball[filtered_df_ball['playing_role'] == 'BOWL']['name_x'])
    all_rounders = set(filtered_df_bat[filtered_df_bat['playing_role'] == 'AR']['name_x']) | \
                   set(filtered_df_ball[filtered_df_ball['playing_role'] == 'AR']['name_x'])

    # Decision variables for each player
    player_vars = {}  
    
    # Filter players with NaN mean batting scores
    batsmen = {player for player in batsmen if not np.isnan(filtered_df_bat[filtered_df_bat['name_x'] == player]['batting_score'].mean())}
    bowlers = {player for player in bowlers if not np.isnan(filtered_df_bat[filtered_df_bat['name_x'] == player]['batting_score'].mean())}
    all_rounders = {player for player in all_rounders if not np.isnan(filtered_df_bat[filtered_df_bat['name_x'] == player]['batting_score'].mean())}

    # Filter players with NaN mean bowling scores
    batsmen = {player for player in batsmen if not np.isnan(filtered_df_ball[filtered_df_ball['name_x'] == player]['bowling_score'].mean())}
    bowlers = {player for player in bowlers if not np.isnan(filtered_df_ball[filtered_df_ball['name_x'] == player]['bowling_score'].mean())}
    all_rounders = {player for player in all_rounders if not np.isnan(filtered_df_ball[filtered_df_ball['name_x'] == player]['bowling_score'].mean())}

    # Create player_vars
    player_vars = {}
    for player in batsmen:
        player_vars[player] = model.addVar(vtype=GRB.BINARY, name=f"bat_{player}")
    for player in bowlers:
        player_vars[player] = model.addVar(vtype=GRB.BINARY, name=f"ball_{player}")
    for player in all_rounders:
        player_vars[player] = model.addVar(vtype=GRB.BINARY, name=f"ar_{player}")


    # Create columns for normalized batting and bowling scores
    
    # Find min and max
    min_bat_score = filtered_df_bat['batting_score'].min()
    max_bat_score = filtered_df_bat['batting_score'].max()
    min_ball_score = filtered_df_ball['bowling_score'].min()
    max_ball_score = filtered_df_ball['bowling_score'].max()

    # Create new normalized column
    filtered_df_bat['batting_score_normalized'] = (filtered_df_bat['batting_score'] - min_bat_score) / (max_bat_score - min_bat_score)
    filtered_df_ball['bowling_score_normalized'] = (filtered_df_ball['bowling_score'] - min_ball_score) / (max_ball_score - min_ball_score)   
        
    # Objective function: Maximize bowling performance
    model.setObjective(
        quicksum(player_vars[player] * filtered_df_bat[filtered_df_bat['name_x'] == player]['batting_score_normalized'].mean() 
                 for player in batsmen | all_rounders) +
        quicksum(player_vars[player] * filtered_df_ball[filtered_df_ball['name_x'] == player]['bowling_score_normalized'].mean()
                 for player in bowlers | all_rounders),
        GRB.MAXIMIZE
    )
    
    
    # Constraints
    # Total team size: 11 players
    model.addConstr(quicksum(player_vars[player] for player in player_vars) == 11, "TotalTeamSize")

    # Minimum number of batsmen, e.g., 4
    min_batsmen = 4
    model.addConstr(quicksum(player_vars[player] for player in batsmen) >= min_batsmen, "MinBatsmen")

    # Minimum number of bowlers, e.g., 4
    min_bowlers = 4
    model.addConstr(quicksum(player_vars[player] for player in bowlers) >= min_bowlers, "MinBowlers")

    # Maximum number of all-rounders, e.g., 2
    max_all_rounders = 2
    model.addConstr(quicksum(player_vars[player] for player in all_rounders) <= max_all_rounders, "MaxAllRounders")

    # Solve the model
    model.optimize()
    
    # Extracting the selected players
    selected_players = [player for player in player_vars if player_vars[player].X > 0.5]
    
    
    # Find average batting score of team for each individual match
    mean_batting_score_per_match = filtered_df_bat.groupby('match_detail_id')['batting_score'].mean()
    
    # Find average batting score of match
    baseline_mean_batting_score = mean_batting_score_per_match.mean()
    print(f"Average team batting score for entire match: {baseline_mean_batting_score}")
    
    # Create df of selected players and their mean batting scores
    selected_players_bat_df = pd.DataFrame(columns=['Player', 'Mean Batting Score'])
    for player in selected_players:
        mean_batting_score = filtered_df_bat[filtered_df_bat['name_x'] == player]['batting_score'].mean()
        selected_players_bat_df = selected_players_bat_df.append({'Player': player, 'Mean Batting Score': mean_batting_score}, ignore_index=True)

    # Find team average batting score
    optimal_mean_batting_score = selected_players_bat_df['Mean Batting Score'].mean()

    print("Optimal team average batting score:", optimal_mean_batting_score)
    
    
    # Find average bowling score of team for each individual match
    mean_bowling_score_per_match = filtered_df_ball.groupby('match_detail_id')['bowling_score'].mean()
    
    # Find average batting score of match
    baseline_mean_bowling_score = mean_bowling_score_per_match.mean()
    print(f"Average team bowling score for entire match: {baseline_mean_bowling_score}")
    
    # Create df of selected players and their mean batting scores
    selected_players_ball_df = pd.DataFrame(columns=['Player', 'Mean Bowling Score'])
    for player in selected_players:
        mean_bowling_score = filtered_df_ball[filtered_df_ball['name_x'] == player]['bowling_score'].mean()
        selected_players_ball_df = selected_players_ball_df.append({'Player': player, 'Mean Bowling Score': mean_bowling_score}, ignore_index=True)

    # Find team average batting score
    optimal_mean_bowling_score = selected_players_ball_df['Mean Bowling Score'].mean()

    print("Optimal team average bowling score:", optimal_mean_bowling_score)

    return selected_players, baseline_mean_batting_score, optimal_mean_batting_score, baseline_mean_bowling_score, optimal_mean_bowling_score

In [25]:
max_both_perf(normalized_df_bat, normalized_df_ball, opponent_name = "Australia", match_type = "test")

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 31 columns and 62 nonzeros
Model fingerprint: 0xe7c28931
Variable types: 0 continuous, 31 integer (31 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 1e+01]
Found heuristic solution: objective 5.4465375
Presolve time: 0.00s
Presolved: 4 rows, 31 columns, 62 nonzeros
Variable types: 0 continuous, 31 integer (31 binary)
Found heuristic solution: objective 6.3246137

Root relaxation: objective 6.668766e+00, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    

C:\Users\PC User\AppData\Local\Temp\ipykernel_39604\1576121378.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  selected_players_bat_df = selected_players_bat_df.append({'Player': player, 'Mean Batting Score': mean_batting_score}, ignore_index=True)
C:\Users\PC User\AppData\Local\Temp\ipykernel_39604\1576121378.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  selected_players_bat_df = selected_players_bat_df.append({'Player': player, 'Mean Batting Score': mean_batting_score}, ignore_index=True)
C:\Users\PC User\AppData\Local\Temp\ipykernel_39604\1576121378.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  selected_players_bat_df = selected_players_bat_df.append({'Player': player, 'Mean Batting Score': mean_bat

(['Karun Nair',
  'Sourav Ganguly',
  'Sachin Tendulkar',
  'Suresh Raina',
  'Shardul Thakur',
  'Mohammed Siraj',
  'Amit Mishra',
  'Mohammed Shami',
  'Jasprit Bumrah',
  'Ravindra Jadeja',
  'Washington Sundar'],
 0.2006692120722943,
 0.2131260910697553,
 0.5201055174098315,
 0.5396680575059452)